In [1]:
import os
import io
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout1D, LSTM, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import joblib
import re

2024-06-13 23:27:18.376536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 23:27:18.376634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 23:27:18.512624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Loading the Dataset

In [2]:
df = pd.read_csv('/kaggle/input/threat-dataset-final/threat_detection_dataset_cleaned.csv')
df.head(10)

,label,type,cleaned_text
0,hate,5.0,መካን ሴት ከማንም መጋባት የለባትም ፍትሃዊ
1,hate,5.0,ብሪተኖች ፋሺስቶች እንዳልሆኑ በመስማቴ ታምኛለሁ
2,hate,5.0,ነገር መለየት እውነት አያደርገውም ሰዎች በቀዶ ሕክምና ራሳቸውን መለወጥ ...
3,hate,5.0,ሴቶች ቢሰበሰቡ በደም ገንዳ እንሰጥም
4,hate,5.0,ቲቪ ጥቁሮችን ጠበኛ ወንጀለኞች በትክክል ይወክላል
5,hate,5.0,መጥቶ ለመለመን ድፍረቱ ዲንዱ ዛሬ አየሁ
6,hate,5.0,ትራንስ ወላጆች ሕፃን አጥፊዎች
7,hate,5.0,ሴቶች የአይጥ እባቦች
8,hate,5.0,ራሴን ሀገር ቤት ለሚመጡ ነገር ያለኝን መስሎ የሚታየኝን መቻቻል እየቀነሰ...
9,hate,5.0,መካን ሴቶች መራባት ባለመቻላቸው የበታች


In [3]:
df.columns = ['label', 'type', 'text']

In [4]:
df

,label,type,text
0,hate,5.0,መካን ሴት ከማንም መጋባት የለባትም ፍትሃዊ
1,hate,5.0,ብሪተኖች ፋሺስቶች እንዳልሆኑ በመስማቴ ታምኛለሁ
2,hate,5.0,ነገር መለየት እውነት አያደርገውም ሰዎች በቀዶ ሕክምና ራሳቸውን መለወጥ ...
3,hate,5.0,ሴቶች ቢሰበሰቡ በደም ገንዳ እንሰጥም
4,hate,5.0,ቲቪ ጥቁሮችን ጠበኛ ወንጀለኞች በትክክል ይወክላል
...,...,...,...
42371,normal,NaN,አይሰቀሉ
42372,normal,NaN,አይሰቀሉ
42373,normal,NaN,አትግደል
42374,normal,NaN,አትጥፋ


In [5]:
df['word_len'] = df['text'].str.split().str.len()
print(df['word_len'].mean())

19.182034882348777


In [6]:
import pandas as pd

# Assuming df is your DataFrame and 'label' column contains the original categorical labels
df['labels'] = df['label'].astype('category').cat.codes

# Get the unique categories and their corresponding codes
label_mapping = dict(enumerate(df['label'].astype('category').cat.categories))

# Print the mapping
print("Label Mapping:", label_mapping)

# To map the numeric codes back to the original labels
df['original_label'] = df['labels'].map(label_mapping)

# Verify the mapping
print(df[['labels', 'original_label']].drop_duplicates())


Label Mapping: {0: 'hate', 1: 'normal', 2: 'threat'}
       labels original_label
0           0           hate
15243       2         threat
20060       1         normal


In [30]:
df['labels'] = df['label'].astype('category').cat.codes
df['labels'].unique()

array([0, 2, 1], dtype=int8)

In [7]:
del df['label']

In [8]:
del df['type']

In [9]:
df["labels"].nunique()

3

In [14]:
del df['original_label']

# Remove NaN values from the dataset

In [15]:
df=df.sample(frac=1)
df

,text,word_len,labels
8568,አቴና ጃዋር ገደል ግብ ምድረ ዶንቆሮ,6.0,0
12917,አንተ መጅመሪያ ሰው ነህ ሰለሰው በደል ፃፋ እንሳሳ ይመስል ሰለ እንሰሳ ...,14.0,0
40959,የኢትዮጵያን ፖለቲካ ጥንቅቅ በመረዳትና ተንትኖ በማስረዳት ወደር የልደቱ ...,24.0,1
32994,ሰለ ቅማንት የሄነ ነገር በለን,5.0,1
3777,ደግ እሰይ ስለመለስ አንተ አይደለህም የምትመሰክረው ሚሊዮኖች ይመሰክሩለታ...,9.0,0
...,...,...,...
38900,በምስራቅ የኦነግ አባላት እርምጃ ተወሰደ በኦሮሚያ ክልል ምስራቅ ሲንቀሳቀ...,84.0,1
39418,ባህር ተፈጠረ ምሽት በሀላ በባህር ከተማ ፍንዳታ የተኩስ ልውውጥ መኮድ አ...,40.0,1
40077,እንደምታዩት የክልላችን ሀይል ሀገሬ ከምትደፈር ልሙትላት ሱዳን ወራሪ ወታ...,26.0,1
5873,ጌጡ ለአንባቢያን ግልፅ አድርገህ እነዲወያዩ በማድረግህ አመሰግናለሁ አስተ...,32.0,0


In [16]:
dataframe = df


In [17]:
dataframe.isna().sum()

text        5
word_len    5
labels      0
dtype: int64

In [18]:
dataframe = dataframe.dropna()


# Data Preprocessing

In [19]:
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

In [20]:
# apply normalization and special characters removal
dataframe['text']=dataframe['text'].apply(lambda x: normalization(x))

/tmp/ipykernel_34/230427076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['text']=dataframe['text'].apply(lambda x: normalization(x))


In [21]:
# special symbol removal
dataframe['text']=dataframe['text'].str.replace('[^\w\s]','',regex=True)

/tmp/ipykernel_34/3924409298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['text']=dataframe['text'].str.replace('[^\w\s]','',regex=True)


In [22]:
# removing digits from the dataset
dataframe['text']=dataframe['text'].str.replace('\d+','',regex=True)

/tmp/ipykernel_34/4246565427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['text']=dataframe['text'].str.replace('\d+','',regex=True)


In [23]:
dataframe


,text,word_len,labels
8568,አቴና ጃዋር ገደል ግብ ምድረ ዶንቆሮ,6.0,0
12917,አንተ መጅመሪያ ሰው ነህ ሰለሰው በደል ፃፋ እንሳሳ ይመስል ሰለ እንሰሳ ...,14.0,0
40959,የኢትዮጵያን ፖለቲካ ጥንቅቅ በመረዳትና ተንትኖ በማስረዳት ወደር የልደቱ ...,24.0,1
32994,ሰለ ቅማንት የሄነ ነገር በለን,5.0,1
3777,ደግ እሰይ ስለመለስ አንተ አይደለህም የምትመሰክረው ሚሊዮኖች ይመሰክሩለታ...,9.0,0
...,...,...,...
38900,በምስራቅ የኦነግ አባላት እርምጃ ተወሰደ በኦሮሚያ ክልል ምስራቅ ሲንቀሳቀ...,84.0,1
39418,ባህር ተፈጠረ ምሽት በሀላ በባህር ከተማ ፍንዳታ የተኩስ ልውውጥ መኮድ አ...,40.0,1
40077,እንደምታዩት የክልላችን ሀይል ሀገሬ ከምትደፈር ልሙትላት ሱዳን ወራሪ ወታ...,26.0,1
5873,ጌጡ ለአንባቢያን ግልፅ አድርገህ እነዲወያዩ በማድረግህ አመሰግናለሁ አስተ...,32.0,0


In [24]:
dataframe['labels'].value_counts()

labels
1    22314
0    18628
2     1429
Name: count, dtype: int64

In [25]:
# Select 10000 samples for label 1
label_1 = dataframe[dataframe['labels'] == 1].sample(n=10000, random_state=42)

# Select 10000 samples for label 0
label_0 = dataframe[dataframe['labels'] == 0].sample(n=10000, random_state=42)

# Select all samples for label 2 - since threat is very small
label_2 = dataframe[dataframe['labels'] == 2]


In [26]:
combined_dataset = pd.concat([label_1, label_0, label_2])


In [27]:
from sklearn.utils import shuffle
# Shuffle the combined dataset
shuffled_dataset = shuffle(combined_dataset, random_state=42)

# Optionally, reset the index
shuffled_dataset = shuffled_dataset.reset_index(drop=True)

shuffled_dataset['labels'] = shuffled_dataset['labels'].replace(2, 0)
# Display the result
print(shuffled_dataset)

                                                    text  word_len  labels
0                                             ቁጥር ከሀዲ ነህ       3.0       1
1      ሰበር ለታሪክ ይቀመጥ የጆነሳይድ ሚድያዎች በትግራይ ያነጣጠረ የዘርማጥፋት...     426.0       1
2      ትናንት መተከል ተገኝቶ ከአሻድሊ ሲመክር አምሽቶ የብአዴንን ተባባሪነት አ...      22.0       1
3      ላለፉት አመታት የተሰሩ ግፎችስ ጌታው የስንቱን እግር ቆረጣቹ ስንቱን ገደ...      31.0       0
4      ሰውዬ የምን አስተማራ ይሆን የኢትዮጵያ ዩኒበርስቲዎቸ ስንቱን መከራ ችለዋ...      13.0       0
...                                                  ...       ...     ...
21424  መልካም ጭንቅላት ታከለ መልካም ይሰራል ክፋ ጭንቅላት ክፋ ይሰራልክፉ ያወ...      24.0       0
21425  ከእንስሳት ያነሰ ሰው መሳይ አውሬ እየኖርን መንግስት ካለ ችግር ስላለ ብ...      26.0       0
21426  በአሁኑ ሰአት በወያኔና በኦነግ እየተደረገ ባለው ከፍተኛ ውጊያ የኦሮሞ ህ...      26.0       1
21427                                      ሚኪ እንኳን አደረሰህ       3.0       1
21428  ጦረኞች ሰው ሲያሰለጥን ኑሮ አሁንም ይህንን አባዜ አለቀቀውም አዬ ኢሱ ሞ...      11.0       0

[21429 rows x 3 columns]


In [28]:
shuffled_dataset["labels"].nunique()

2

# Split the Dataset

In [29]:
# train test split
from sklearn.model_selection import train_test_split
train_val_df, test_dataset = train_test_split(shuffled_dataset, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (15428, 3)
Validation dataset shape:  (1715, 3)
Testing dataset shape:  (4286, 3)


In [36]:
data_point = test_dataset.iloc[2147]
data_point

text        ድድብና መልክ ቢኖረው ቁጭ አንተን ሚመስለው ምን ያረጋል ምን አለ ትንሽ ...
word_len                                                 21.0
labels                                                      0
Name: 3018, dtype: object

In [55]:
from tensorflow.keras.utils import to_categorical
# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer( )# You can adjust the minimum document frequency

# Fit and transform the training data
train_vectors = vectorizer.fit_transform(train_dataset['text'])

# Transform the validation and test data
valid_vectors = vectorizer.transform(evaluation_dataset['text'])
test_vectors = vectorizer.transform(test_dataset['text'])

# Label encoding for the target variable
le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_dataset['labels'])
valid_labels_encoded = le.transform(evaluation_dataset['labels'])
test_labels_encoded = le.transform(test_dataset['labels'])

# Print the vocabulary (optional)
print("Vocabulary:", len(vectorizer.get_feature_names_out()))

# Convert sparse matrices to dense NumPy arrays (optional)
train_vectors_dense = train_vectors.toarray()
valid_vectors_dense = valid_vectors.toarray()
test_vectors_dense = test_vectors.toarray()
input_shape = train_vectors.shape[1]
print(input_shape)


Vocabulary: 79704
79704


In [40]:
joblib.dump(vectorizer, 'vectorizer_m.pkl')

['vectorizer_m.pkl']

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_labels_encoded), y=train_labels_encoded)
class_weights = dict(enumerate(class_weights))

# Define the neural network model
cnn_model_2 = Sequential()
cnn_model_2.add(Input(shape=(input_shape,)))
cnn_model_2.add(Dense(32, activation='relu', kernel_regularizer=l2(0.05)))  # Increased regularization
cnn_model_2.add(BatchNormalization())
cnn_model_2.add(Dropout(0.5))
cnn_model_2.add(Dense(16, activation='relu', kernel_regularizer=l2(0.05)))  # Increased regularization
cnn_model_2.add(BatchNormalization())
cnn_model_2.add(Dropout(0.5))
cnn_model_2.add(Dense(1, activation='sigmoid'))

# Compile the model
cnn_model_2.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])  # Lower learning rate

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
cnn_model_2.fit(train_vectors_dense, train_labels_encoded, 
                epochs=20, batch_size=32, 
                validation_data=(valid_vectors_dense, valid_labels_encoded), 
                callbacks=[early_stopping],
                class_weight=class_weights)

# Evaluate the model on the test set
predictions = (cnn_model_2.predict(test_vectors_dense) > 0.5).astype(int)
accuracy = accuracy_score(test_labels_encoded, predictions)
print("Test Accuracy:", accuracy)

# Debugging if predictions are equal to test_labels_encoded
print(np.all(predictions == test_labels_encoded))


Epoch 1/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.5422 - loss: 2.4822 - val_accuracy: 0.6187 - val_loss: 1.6359
Epoch 2/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6692 - loss: 1.5229 - val_accuracy: 0.6746 - val_loss: 1.3527
Epoch 3/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7452 - loss: 1.2373 - val_accuracy: 0.6828 - val_loss: 1.1859
Epoch 4/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8226 - loss: 0.9796 - val_accuracy: 0.6723 - val_loss: 1.0763
Epoch 5/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8702 - loss: 0.7754 - val_accuracy: 0.6816 - val_loss: 1.0078
Epoch 6/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9000 - loss: 0.6209 - val_accuracy: 0.6840 - val_loss: 0.9848
Epoch 7/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9106 - loss: 0.5297 - val_accuracy: 0.6706 - val_loss: 0.9871
Epoch 8/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9205 - loss: 0.4621 - val_accuracy: 

In [57]:
cnn_model_2.input_shape

(None, 79704)

In [59]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SpatialDropout1D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import random

# Reshape input data to add timestep dimension
train_vectors_reshaped = np.reshape(train_vectors_dense, (train_vectors_dense.shape[0], 1, train_vectors_dense.shape[1]))
valid_vectors_reshaped = np.reshape(valid_vectors_dense, (valid_vectors_dense.shape[0], 1, valid_vectors_dense.shape[1]))
test_vectors_reshaped = np.reshape(test_vectors_dense, (test_vectors_dense.shape[0], 1, test_vectors_dense.shape[1]))

# Model parameters
lstm_out = 32  # Adjusted number of LSTM units
dense_units = 16  # Adjusted number of Dense units

# Initialize the model
model_lstm = Sequential()

# Add LSTM layer with adjusted dropout and regularization
model_lstm.add(SpatialDropout1D(0.3))  # Adjusted dropout rate
model_lstm.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3, kernel_regularizer=l2(0.01)))  # Adjusted dropout and regularization
model_lstm.add(BatchNormalization())

# Add Dense output layer with adjusted dropout and regularization
model_lstm.add(Dense(dense_units, activation='relu', kernel_regularizer=l2(0.01)))  # Adjusted units and regularization
model_lstm.add(Dense(1, activation='sigmoid'))

# Compile the model
model_lstm.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])  # Adjusted learning rate

# Print model summary
print(model_lstm.summary())

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model_lstm.fit(train_vectors_reshaped, train_labels_encoded, 
               epochs=50, batch_size=64, 
               validation_data=(valid_vectors_reshaped, valid_labels_encoded), 
               callbacks=[early_stopping])

# Predict probabilities for each class
predictions_prob = model_lstm.predict(test_vectors_reshaped)

# Convert probabilities to class labels
predictions = (predictions_prob > 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(test_labels_encoded, predictions)
print("Accuracy:", accuracy)

# Get a random sample from the test dataset
random_index = random.randint(0, len(test_labels_encoded) - 1)
predicted_class = predictions[random_index]
expected_class = test_labels_encoded[random_index]

# Print the predicted class and the expected class for the random sample
print(f"Random Test Sample Index: {random_index}")
print(f"Predicted Class: {predicted_class}")
print(f"Expected Class: {expected_class}")


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spatial_dropout1d_4             │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.6080 - loss: 1.1350 - val_accuracy: 0.5761 - val_loss: 0.8696
Epoch 2/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7298 - loss: 0.7508 - val_accuracy: 0.6752 - val_loss: 0.8272
Epoch 3/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7685 - loss: 0.6936 - val_accuracy: 0.6682 - val_loss: 0.7966
Epoch 4/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8017 - loss: 0.6362 - val_accuracy: 0.6770 - val_loss: 0.7728
Epoch 5/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8285 - loss: 0.5826 - val_accuracy: 0.6886 - val_loss: 0.7787
Epoch 6/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8482 - loss: 0.5298 - val_accuracy: 0.6741 - val_loss: 0.7996
Epoch 7/50
242/242 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8574 - loss: 0.4978 - val_accuracy: 0.6799 - val_loss: 0.7883
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Accuracy: 0.6847876808212786
Random Test Sample Ind

In [63]:
model_lstm.input_shape

(None, 1, 79704)

In [62]:
predictions_prob = model_lstm.predict(test_vectors_reshaped)

# Convert probabilities to class labels
predictions = (predictions_prob > 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(test_labels_encoded, predictions)
print("Accuracy:", accuracy)

# Get a random sample from the test dataset
random_index = random.randint(0, len(test_labels_encoded) - 1)
predicted_class = predictions[random_index]
expected_class = test_labels_encoded[random_index]

# Print the predicted class and the expected class for the random sample
print(f"Random Test Sample Index: {random_index}")
print(f"Predicted Class: {predicted_class}")
print(f"Expected Class: {expected_class}")

134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Accuracy: 0.6847876808212786
Random Test Sample Index: 519
Predicted Class: 0
Expected Class: 0


In [47]:
data_point = test_dataset.iloc[385]
data_point

text        የኢትዮጵያ ህዝብ ሞኝ ተላላ ህዝብ እኮ ነውኮረቻው ካልወጣው አላምንም ያለ...
word_len                                                 11.0
labels                                                      1
Name: 11952, dtype: object

In [64]:
joblib.dump(model_lstm, 'lstm_.pkl')

['lstm_.pkl']